# Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

In [7]:
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)

In [8]:
cv_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score',ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,271.432024,2.158446,0.234101,0.044407,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.971300,0.981132,0.970350,0.968553,0.968553,0.971978,0.004697,1
3,481.212988,73.287219,0.252590,0.031685,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964126,0.979335,0.969452,0.968553,0.968553,0.970002,0.005023,2
5,5292.622444,1448.122982,0.240447,0.089409,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965919,0.979335,0.970350,0.969452,0.964960,0.970002,0.005091,2
4,453.217512,36.319653,0.252083,0.042377,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.962332,0.974843,0.969452,0.968553,0.968553,0.968744,0.003973,4
2,418.691017,97.150161,0.238134,0.027093,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.963229,0.977538,0.967655,0.968553,0.964960,0.968385,0.004953,5


In [9]:
cv_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score',ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,793.121307,720.649899,0.402185,0.113637,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965022,0.976640,0.967655,0.965858,0.971249,0.969283,0.004255,1
0,173.548362,4.942949,0.207587,0.018360,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965022,0.978437,0.970350,0.962264,0.968553,0.968924,0.005517,2
2,1361.598761,903.490700,0.234690,0.025559,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964126,0.975741,0.966757,0.964061,0.972147,0.968565,0.004639,3
5,371.804486,65.214086,0.156266,0.023053,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.962332,0.977538,0.967655,0.964960,0.970350,0.968565,0.005223,3
1,229.678524,1.786276,0.206482,0.004352,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964126,0.977538,0.971249,0.964061,0.964960,0.968385,0.005302,5
